## algorithm design and anlysis-2025 spring  homework 4
**Deadline**：2025.5.14

**name**:


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1 
**最小生成树（Minimum Spanning Tree）**

设  **G**  为一个带权重的连通无向图，且所有边的权重均不相等。令$e_i$ 为权重第 $i$ 小的边。最小生成树（MST）是否必须包含 $e_1$ ? 同理，是否必须包含 $e_2$ 和 $e_3$ ? 若必须包含，请给出证明；否则，请构造反例。需从基本原理论证，不能依赖割引理(cut lemma) 或 Prim/Kruskal算法的正确性。


answer:  
![figure](./fig/hw4q1_a1.png)

## 问题 2 
**瓶颈生成树（Bottleneck Spanning Tree）**

带有权重的无向图 $G(V,E,w)$ 的瓶颈生成树，表现为：在所有生成树中，最大权重边的权重值最小。即，BST $T$ 最小化瓶颈损失 $c(T)=max_{e \in T}{w(e)}$。

1. 证明 $G$ 的每一个最小生成树（MST）都是瓶颈生成树（BST）
2. 设计一个线性时间复杂度的算法：， 对于一个图 $G(V,E,w)$ 和一个整数 $b$，判断图 $ G$ 是否存在一个瓶颈生成树，其最大权重边的权重不超过 $b$，分析算法设计思路，并基于python编程实现。
3. 设计一个线性时间复杂度的算法：对于给定的图 $G(V,E,w)$，找到其瓶颈生成树，分析算法设计思路，并基于python编程实现。

idea：    
1.![figure](./fig/hw4q2_a2.png)  
2.算法思路：把所有权重大于b的边删掉，留下的子图如果一次BFS能访问到全部顶点，就说明它连通，能从中抽出一棵最大边权不超过b的生成树；若有顶点未被访问到，则无此生成树。

In [1]:
from collections import deque
def exists_bottleneck_tree(vertices, edges, b):
    adj = {v: [] for v in vertices}
    for u, v, w in edges:
        if w <= b:
            adj[u].append(v)
            adj[v].append(u)
    start = next(iter(vertices))
    visited = {start}
    queue = deque([start])
    while queue:
        u = queue.popleft()
        for nbr in adj[u]:
            if nbr not in visited:
                visited.add(nbr)
                queue.append(nbr)
    return len(visited) == len(vertices)

V = ['A', 'B', 'C', 'D']
E = [('A','B',1), ('A','C',2), ('B','C',3), ('C','D',4)]
for b in range(1, 6):
    print(f"b = {b}: ", exists_bottleneck_tree(V, E, b))
# algorithm of the liear time complexity 

b = 1:  False
b = 2:  False
b = 3:  False
b = 4:  True
b = 5:  True


idea：  
3.算法思路：把所有边权去重装入数组，在这上面随机选枢轴p，用并查集判断只保留权≤p的边能否连通：能就继续在≤p的部分划分，不能就在>p的部分划分，直到缩到唯一值b。然后再在所有权≤b的边里依次用并查集合并端点、取边，直到凑齐|V|–1条边，这棵树的最大边权即b。

In [2]:
import random
from collections import deque
class UnionFind:
    def __init__(self, vertices):
        self.parent = {v: v for v in vertices}
        self.rank = {v: 0 for v in vertices}
        self.count = len(vertices)

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        rootX = self.find(x)
        rootY = self.find(y)
        if rootX == rootY:
            return False
        if self.rank[rootX] < self.rank[rootY]:
            self.parent[rootX] = rootY
        elif self.rank[rootX] > self.rank[rootY]:
            self.parent[rootY] = rootX
        else:
            self.parent[rootY] = rootX
            self.rank[rootX] += 1
        self.count -= 1
        return True

def is_connected_threshold(vertices, edges, threshold):
    uf = UnionFind(vertices)
    for u, v, w in edges:
        if w <= threshold:
            uf.union(u, v)
    return uf.count == 1

def find_bottleneck_tree(vertices, edges):
    weights = list({w for _, _, w in edges})
    lo, hi = 0, len(weights)
    while hi - lo > 1:
        pivot_idx = random.randrange(lo, hi)
        pivot = weights[pivot_idx]
        weights[pivot_idx], weights[hi-1] = weights[hi-1], weights[pivot_idx]
        store = lo
        for i in range(lo, hi-1):
            if weights[i] <= pivot:
                weights[store], weights[i] = weights[i], weights[store]
                store += 1
        weights[store], weights[hi-1] = weights[hi-1], weights[store]
        if is_connected_threshold(vertices, edges, weights[store]):
            hi = store + 1
        else:
            lo = store + 1
    b = weights[lo]
    uf = UnionFind(vertices)
    tree_edges = []
    for u, v, w in edges:
        if w <= b and uf.union(u, v):
            tree_edges.append((u, v, w))
        if len(tree_edges) == len(vertices) - 1:
            break
    return b, tree_edges

V = ['A', 'B', 'C', 'D']
E = [('A','B',1), ('A','C',2), ('B','C',3), ('C','D',4)]
bottleneck, tree = find_bottleneck_tree(V, E)
print("Bottleneck weight:", bottleneck)
print("Tree edges:", tree)

Bottleneck weight: 4
Tree edges: [('A', 'B', 1), ('A', 'C', 2), ('C', 'D', 4)]


## 问题 3

**道路网（Road Network）**

假设有一个以图 $ G(V, E, l) $ 表示的道路网络，连接了一组城市 $ V $。我们假设该网络是有向的，并且每条道路 $(u, v) \in E$ 都有一个非负的长度 $ l(u, v) $。一条新的道路即将被建造，因此有一个列表 $ E' $ 包含它可以连接的城市对。每对 $(u, v) \in E'$ 都有一个对应的长度 $ l'(u, v) $。我们希望选择一对城市，使得两个城市 $ s, t \in V $ 之间的距离减少最大。请为此问题编写一个高效的算法，并详细解释算法的正确性和复杂度。

答：将“是否已经使用过新路”也看成一个状态，把每个原始顶点v拆成两份：(v,0)表示“还没用过新路”，(v,1)表示“已经用了新路”。在这个二层图中，任何原有道路(u,v)都在各自层内以相同长度连通(u,0)→(v,0)、(u,1)→(v,1)，而每条候选新路(u,v)则只在0层跨到1层，即只允许一次从(u,0) 到(v,1)使用它。这样，一次从(s,0)出发跑普通的Dijkstra，就能在(t,0) 与(t,1) 两个目标状态上各自找到最短路，较小者恰对应“允许最多用一条新路”的全局最优路径。
正确性：任何在分层图中走到(t,0)的路径绝不使用新路，恰对应原图不加新边的路径；走到(t,1)且跨过一次“0→1”边的路径，则恰使用了且只使用了一条新路，且路径长度正是原图中“最短(s→u)+新路(u→v)+最短(v→t)”的组合。因此分层最短路完备且无冗余。
复杂度：分层图有$2∣V∣$个节点、约$2∣E∣+∣E'∣$条边，Dijkstra 的时间是$O((∣V∣+∣E∣+∣E'∣)log∣V∣)$，相比为每条新路都重跑一次最短路要高效得多。

## 问题 4

**逃离问题**

一个 $ n \times n $ 的网格是一个无向图，由 $ n $ 行和 $ n $ 列的顶点组成，如下图所示。我们用 $(i,j)$ 表示第 $ i $ 行和第 $ j $ 列的顶点。除了边界顶点，网格中的所有顶点都有四个邻居，即满足 $ i = 1, i = n, j = 1 $ 或 $ j = n $ 的点 $(i,j)$。

给定网格中的 $ m \leq n^2 $ 个起点 $(x_1, y_1), (x_2, y_2), \cdots , (x_m, y_m)$，逃离问题是确定是否存在 $ m $ 条顶点不相交的路径（即路径之间不相交），从这些起点到边界上的任意 $ m $ 个不同点。例如，图1中的网格存在逃离。

(1) 该问题可以看作是一个最大流问题。考虑一个流网络，其中顶点和边都有容量。也就是说，进入任何给定顶点的总正流量受到容量限制。证明在具有边和顶点容量的网络中确定最大流可以简化为在具有可比大小的普通流网络上的最大流问题。更准确地说，你需要将一个具有顶点和边容量的网络 $ G = (V,E) $ 转换为另一个仅具有边容量的网络 $ G' = (V', E') $，使得两个网络上的最大流相同，并且你构建的新网络具有 $ V' = O(V) $ 个顶点和 $ E' = O(E) $ 条边。你可以假设网络是连通的。

(2) 描述一个解决逃离问题的高效算法，并分析其运行时间。


<div align="center"> <img alt="图片" src="./fig/escepe-p.png"> </div>
<center> 图2. 逃脱问题网格，起始顶点为黑色，其他网格顶点为白色</center>

idea：  
1.对原网络G=(V,E) 中的每个顶点v，在新网络中引入两个顶点$v_{\text{in}}$和$v_{\text{out}}$，并在它们之间增加一条有向边$v_{\mathrm{in}}\longrightarrow v_{\mathrm{out}}$，其容量设为原顶点的容量c(v)。原网络中任意有向边(u→v)在新网络中变为$v_{\mathrm{out}}\longrightarrow v_{\mathrm{in}}$,
容量保持为c(u,v)。如果原来是一条无向边{u,v}，则在两端各加入一条上述形式的有向边，容量同为c(u,v)。源点拆分后取其“输出端”作为新网络的源，汇点拆分后取其“输入端”作为新网络的汇。这样得到的网络顶点数为$∣V'∣=2∣V∣$,边数为
$∣E'∣=∣V∣+2∣E∣$，仍然满足$O(∣V∣)$和$O(∣E∣)$的规模要求。  
等价性证明:任取原网络上的可行流f。对每条原边(u→v)定义新网络中对应边的流量为$f'(\,u_{\mathrm{out}}\to v_{\mathrm{in}})=f(u\to v)$，对每条拆分边定义$f'\bigl(v_{\mathrm{in}}\to v_{\mathrm{out}}\bigr)\;=\;\sum_{u}f(u\to v)$
其中右侧等于流入顶点v的总流量。这样既满足新网络的边容量约束，也保证流守恒；反过来，新网络上的任何可行流也可按上述方式压缩回原网络，因而最大流值保持一致。  
2.首先将每个网格顶点 $v$ “拆分”为两个节点 $v_{\text{in}}$ 和 $v_{\text{out}}$，并在它们之间引入一条容量为 1 的有向边 $v_{\text{in}}\to v_{\text{out}}$；原来每对相邻顶点 $u,v$ 之间的无向边，则分别在新网络中加入两条有向边 $u_{\text{out}}\to v_{\text{in}}$ 和 $v_{\text{out}}\to u_{\text{in}}$（容量同为 1）。接着增加超级源 $S$，将其与每个起点 $(x_k,y_k)$ 对应的 $(x_k,y_k)_{\text{in}}$ 以单位容量相连；再增加超级汇 $T$，将每个边界顶点 $(i,j)_{\text{out}}$ 与 $T$ 以容量 1 相连。最后在该单位容量网络上运行 Dinic 算法求最大流 $f_{\max}$，若 $f_{\max}=m$ 则存在 $m$ 条顶点不相交的路径，否则不存在。  
构造上述流网络及其所有边耗时与网格规模成正比，为 $O(n^2)$；而 Dinic 算法在单位容量图上的时间复杂度为 $O(E\sqrt{V})$，此处 $V=O(n^2)$、$E=O(n^2)$，所以总时间复杂度为 $O(n^2\cdot n)=O(n^3)$。









